In [8]:
search_for = 84
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0038301944732666016
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 220637571
type: [1, 1, 1, 1, 84] 84
cases of this type: 49787136
10000 31.988436559196934
20000 34.47640912262223
30000 124.66787930940734
40000 122.2624845433867
50000 121.9543735842214
60000 134.2701991487662
70000 128.1499099923822
80000 128.0677626002282
90000 124.58579669706312
100000 122.1817403613758
110000 116.85349702664658
120000 125.27239461154956
130000 124.66445092638996
140000 120.34337380006342
150000 136.93205519405657
160000 139.52715107879072
170000 125.56944358949917
180000 122.74201413993612
190000 124.53667438037989
200000 117.56840998891762
210000 118.07370903492372
220000 120.19403434709866
230000 121.67330374213557
240000 123.99097999502779
250000 125.08683270736006
260000 118.17025123665705
270000 117.8206181500293
280000 131.2015272617856
290000 50.689791458905916
300000 19.413834644459286
310000 19.5

3050000 99.01465185481999
3060000 104.47089759859035
3070000 90.98394040934399
3080000 101.81033445488636
3090000 98.71142967950897
3100000 102.93925762693709
3110000 108.46520954768178
3120000 103.43560369485259
3130000 103.7885105425889
3140000 104.10449942490912
3150000 99.66756538409524
3160000 100.45336010946009
3170000 99.93236361250986
3180000 91.98932334894175
3190000 101.98053069659933
3200000 109.51497926530845
3210000 102.23790315921754
3220000 98.26369277178269
3230000 90.79287796274565
3240000 96.60984193374425
3250000 76.24698360723364
3260000 18.49466537145175
3270000 18.39837692720413
3280000 19.029341029750764
3290000 103.77801435351192
3300000 93.26754385983807
3310000 90.98406769709753
3320000 100.66090007693491
3330000 103.80305905411649
3340000 106.79526293951413
3350000 92.27877382757279
3360000 94.1993541430464
3370000 98.07840656284554
3380000 97.78263231437259
3390000 100.38058561362215
3400000 108.68352734125662
3410000 108.03104367773379
3420000 105.519245449

6160000 91.93317059332001
6170000 94.23561557212092
6180000 90.8333699347297
6190000 90.52417552340575
6200000 89.81655132313061
6210000 106.27521453549592
6220000 18.722453084567615
6230000 18.82234206035829
6240000 19.15867315891248
6250000 74.71013841908422
6260000 103.27186680753942
6270000 87.57452117423439
6280000 96.25656084308044
6290000 93.58048623216584
6300000 92.14991303492418
6310000 92.05727802188665
6320000 87.22183195021289
6330000 93.17992749935704
6340000 102.06583313762289
6350000 102.18299410540835
6360000 99.30095336736274
6370000 100.81216539232426
6380000 106.48939821538318
6390000 96.20182494492185
6400000 98.47592548644552
6410000 108.6228976021632
6420000 99.65331751674313
6430000 97.93642440503345
6440000 77.94793151175386
6450000 45.90409806366527
6460000 45.62361969011757
6470000 43.613883560474406
6480000 111.96583911327475
6490000 102.30852906677505
6500000 98.3465865142643
6510000 103.82116013484372
6520000 109.13550620304032
6530000 116.63397095443607
6

9300000 89.66722860543558
9310000 86.47475348556388
9320000 85.27880449530677
9330000 97.86095615182059
9340000 90.34744641544985
9350000 91.98339633923905
9360000 83.27873015428361
9370000 62.10913501854929
9380000 42.55194399765166
9390000 41.08935713545272
9400000 53.568401751459724
9410000 110.9545485088098
9420000 89.14319226186454
9430000 90.33577693099677
9440000 92.49993649168005
9450000 84.79394685606731
9460000 92.46494108153978
9470000 89.97450976971096
9480000 94.5686960522125
9490000 103.57448805638322
9500000 99.03772099482791
9510000 92.26471371789717
9520000 100.02948849891908
9530000 97.39220677051216
9540000 99.14462987513294
9550000 99.83734939581188
9560000 90.35304624544918
9570000 96.04548146839748
9580000 94.38593404137998
9590000 46.442858189248604
9600000 35.31898816091012
9610000 31.72468327138239
9620000 52.23773037078635
9630000 105.20022347730044
9640000 91.81643021773615
9650000 92.39730653475333
9660000 90.4442102879025
9670000 91.76298722930804
9680000 8

12360000 86.39415058509184
12370000 104.45484553828625
12380000 98.32289411118305
12390000 89.03636578674835
12400000 86.94182533003232
12410000 86.72249289558061
12420000 101.75628529689854
12430000 94.04250855312584
12440000 100.31239699555371
12450000 119.86652679601971
12460000 109.72403497213934
12470000 104.67166351171839
12480000 99.05512305899924
12490000 85.1264150419577
12500000 93.10463950433679
12510000 94.20915535088376
12520000 94.96523030851894
12530000 96.22825432035026
12540000 95.84693908683391
12550000 89.21789715343334
12560000 99.85354259421132
12570000 87.8640871597327
12580000 96.45045635436848
12590000 44.99756963358245
12600000 34.351459734688206
12610000 35.440971708912194
12620000 54.145457048927376
12630000 95.5749414465374
12640000 79.95025569826494
12650000 74.8424215905331
12660000 87.89379482032781
12670000 82.87831761179697
12680000 92.6172973578334
12690000 83.7656480336475
12700000 79.39761734384206
12710000 91.67060913336289
12720000 76.5403556625516

...

211170000 0.903100894075811
211180000 0.5163992268672719
211190000 0.9743210161347322
211200000 0.9667765189415217
211210000 0.7394485698647697
211220000 0.7426052654051715
211230000 0.9952461801538468
211240000 0.9888190795709357
211250000 0.5230185518770285
211260000 0.8761254907019933
211270000 0.9894777930105659
211280000 0.7910946567979985
211290000 0.6514056250185967
211300000 0.9387366408686306
211310000 0.9965127254301442
211320000 0.7947687683345477
211330000 0.6389585291666587
211340000 0.9772182080614898
211350000 0.9977124026033085
211360000 0.6457879971557988
211370000 0.9067847912034657
211380000 1.057060681839466
211390000 1.0502491010961004
211400000 0.4093640023563836
211410000 1.0574743453947306
211420000 1.0657377306987246
211430000 0.8934557134496437
best so far: 0
type: [2, 1, 1, 2, 21] 84
cases of this type: 1555848
211440000 0.6615649986216028
211450000 0.7217239917375313
211460000 0.7153781123667492
211470000 0.33210449739313125
211480000 0.4228554781346056
2114

213930000 0.26532374538769327
213940000 0.3960783812115656
213950000 0.39453724959245656
213960000 0.3944631200781861
213970000 0.38722332017537625
213980000 0.37169414113979876
213990000 0.23993599647289515
214000000 0.23973731091002623
214010000 0.3724871854063471
214020000 0.3764141695762078
best so far: 0
type: [2, 1, 1, 6, 7] 84
cases of this type: 518616
214030000 0.3876935247537693
214040000 0.3608320164978637
214050000 0.29903844532990453
214060000 0.26874872985982234
214070000 0.2687268630598386
214080000 0.2713238079419732
214090000 0.3198938100002872
214100000 0.378250719274044
214110000 0.37739866389564675
214120000 0.3103320169602831
214130000 0.2612538494821654
214140000 0.2619540859930118
214150000 0.25858519348329967
214160000 0.2767058313674463
214170000 0.3650561078339219
214180000 0.37408686966612603
214190000 0.34060758792506984
214200000 0.2846710342962543
214210000 0.26124817637740905
214220000 0.26188085586222015
214230000 0.2704180984343092
214240000 0.336489945

216530000 0.16994128857604662
216540000 0.17088137927238148
216550000 0.17398562444098792
216560000 0.1736313523148497
216570000 0.16976837305772305
216580000 0.16928113485854204
best so far: 0
type: [2, 1, 2, 21, 1] 84
cases of this type: 37044
216590000 0.11682211807779472
216600000 0.11330648563766481
216610000 0.11321867839796676
best so far: 0
type: [2, 1, 3, 1, 14] 84
cases of this type: 345744
216620000 0.14499069097133477
216630000 0.2564415363866488
216640000 0.253716297884908
216650000 0.24091936061851182
216660000 0.23190937395989894
216670000 0.22451706826731893
216680000 0.22183137895484764
216690000 0.2149776669970155
216700000 0.22406614028371705
216710000 0.22969498223465679
216720000 0.22220612612936894
216730000 0.23920673348599011
216740000 0.24052303007278178
216750000 0.13777724600028993
216760000 0.12581321068063045
216770000 0.12509798088342614
216780000 0.12486677995580435
216790000 0.12459206255508132
216800000 0.12398399859193299
216810000 0.12365148940267165


best so far: 0
type: [2, 7, 1, 6, 1] 84
cases of this type: 1512
best so far: 0
type: [2, 7, 2, 1, 3] 84
cases of this type: 2268
best so far: 0
type: [2, 7, 2, 3, 1] 84
cases of this type: 756
best so far: 0
type: [2, 7, 3, 1, 2] 84
cases of this type: 1008
best so far: 0
type: [2, 7, 3, 2, 1] 84
cases of this type: 504
best so far: 0
type: [2, 7, 6, 1, 1] 84
cases of this type: 252
best so far: 0
type: [2, 14, 1, 1, 3] 84
cases of this type: 1134
best so far: 0
type: [2, 14, 1, 3, 1] 84
cases of this type: 378
best so far: 0
type: [2, 14, 3, 1, 1] 84
cases of this type: 126
best so far: 0
type: [2, 21, 1, 1, 2] 84
cases of this type: 336
best so far: 0
type: [2, 21, 1, 2, 1] 84
cases of this type: 168
best so far: 0
type: [2, 21, 2, 1, 1] 84
cases of this type: 84
best so far: 0
type: [2, 42, 1, 1, 1] 84
cases of this type: 42
218400000 0.06699627483737469
best so far: 0
type: [3, 1, 1, 1, 28] 84
cases of this type: 614656
218410000 0.2784995286891262
218420000 0.2449166537775861
218

best so far: 0
type: [3, 7, 4, 1, 1] 84
cases of this type: 112
best so far: 0
type: [3, 14, 1, 1, 2] 84
cases of this type: 224
best so far: 0
type: [3, 14, 1, 2, 1] 84
cases of this type: 112
best so far: 0
type: [3, 14, 2, 1, 1] 84
cases of this type: 56
best so far: 0
type: [3, 28, 1, 1, 1] 84
cases of this type: 28
best so far: 0
type: [4, 1, 1, 1, 21] 84
cases of this type: 194481
220140000 0.024198182244300842
220150000 0.03215292906747924
220160000 0.029496873244100146
220170000 0.030130828938901424
220180000 0.030103073757456404
220190000 0.028344707582142623
220200000 0.02882563025039434
220210000 0.030450629936754705
220220000 0.013752494911511738
220230000 0.013251251693964005
220240000 0.012830222175697487
220250000 0.012505995098802777
220260000 0.019018963104486464
220270000 0.024270948888878027
220280000 0.022120397673732705
220290000 0.0218388749717474
220300000 0.02232409863188532
220310000 0.020897623091002304
220320000 0.020801001806437968
220330000 0.02161818863487